In [1]:
!git clone https://github.com/alvaromoure/yolov5.git
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 10485, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 10485 (delta 0), reused 1 (delta 0), pack-reused 10481
Receiving objects: 100% (10485/10485), 12.29 MiB | 34.56 MiB/s, done.
Resolving deltas: 100% (7054/7054), done.
/kaggle/working/yolov5
Note: you may need to restart the kernel to use updated packages.


In [2]:
%cd ..

/kaggle/working


!pip install -q --upgrade wandb

# Imports

In [3]:
import torch
import wandb
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt
import json

In [5]:
FOLD = 0
EPOCHS = 300

In [6]:
PROJECT_NAME = 'Final-Covid19-Detection'
NOTEBOOK_NAME = ''
EXP_NAME = F'yoloV5x_300e_fold{FOLD}'

In [10]:
train_ann_file = f'/kaggle/working/coco_train_512x512_fold_{FOLD}.json'
val_ann_file = f'/kaggle/working/coco_val_512x512_fold_{FOLD}.json'

## Creating YoloV5 folder structure using annotations files

In [11]:
os.makedirs('/kaggle/working/dataset')
os.makedirs('/kaggle/working/dataset/images')
os.makedirs('/kaggle/working/dataset/images/train')
os.makedirs('/kaggle/working/dataset/images/val')
os.makedirs('/kaggle/working/dataset/labels')
os.makedirs('/kaggle/working/dataset/labels/train')
os.makedirs('/kaggle/working/dataset/labels/val')



In [12]:
with open(train_ann_file,'rb') as file:
    train_dict = json.load(file)
train_image_paths = []
for item in tqdm(train_dict['images']):
    train_image_paths.append(item['file_name'])
                            
                             
with open(val_ann_file,'rb') as file:
    val_dict = json.load(file)
val_image_paths = []
for item in tqdm(val_dict['images']):
    val_image_paths.append(item['file_name'])
    
    
                             

100%|██████████| 859/859 [00:00<00:00, 1424073.97it/s]


In [13]:
for image in tqdm(train_image_paths):
    image_id = image.split('_')[0]
    shutil.copyfile(os.path.join('/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512',
                                 image),f'/kaggle/working/dataset/images/train/{image_id}.png')
for image in tqdm(val_image_paths):
    image_id = image.split('_')[0]
    shutil.copyfile(os.path.join('/kaggle/input/siim-covid19-images-metadata-256-512-768/images_metadata_256_512_768/train_512x512',
                                 image),f'/kaggle/working/dataset/images/val/{image_id}.png')
    

100%|██████████| 859/859 [00:03<00:00, 234.93it/s]


### Creating the data YAML file

In [14]:
import yaml
data_yaml = dict(
    path = '/kaggle/working/dataset',
    train = 'images/train/',
    val = 'images/val/',
    nc=1,
    names = ['Covid_Opacity',]
)

with open('/kaggle/working/yolov5/data/data.yaml','w') as file:
    yaml.dump(data_yaml,file,default_flow_style=True)

In [15]:
IMG_SIZE = 512

def get_bbox(row):
    bboxes = []
    bbox = []
    for i, l in enumerate(row.label.split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l))
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []  
            
    return bboxes

# Scale the bounding boxes according to the size of the resized image. 
def scale_bbox(row, bboxes):
    # Get scaling factor
    scale_x = IMG_SIZE/row.dim1
    scale_y = IMG_SIZE/row.dim0
    
    scaled_bboxes = []
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x, 4))
        y = int(np.round(bbox[1]*scale_y, 4))
        x1 = int(np.round(bbox[2]*(scale_x), 4))
        y1= int(np.round(bbox[3]*scale_y, 4))

        scaled_bboxes.append([x, y, x1, y1]) # xmin, ymin, xmax, ymax
        
    return scaled_bboxes

# Convert the bounding boxes in YOLO format.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0] # xmax - xmin
        h = bbox[3] - bbox[1] # ymax - ymin
        xc = bbox[0] + int(np.round(w/2)) # xmin + width/2
        yc = bbox[1] + int(np.round(h/2)) # ymin + height/2
        
        yolo_boxes.append([xc/img_w, yc/img_h, w/img_w, h/img_h]) # x_center y_center width height
    
    return yolo_boxes

In [16]:
df = pd.read_csv('/kaggle/input/siim-covid19-detection/train_image_level.csv')
df['id'] = df.apply(lambda row: row.id.split('_')[0], axis=1)
df['image_level'] = df.apply(lambda row: row.label.split(' ')[0], axis=1)
meta_df = pd.read_csv(f'/kaggle/input/siim-covid-19-256-512-768-1024-1280/train_meta_{IMG_SIZE}x{IMG_SIZE}.csv')
meta_df['id'] = meta_df.apply(lambda row: row.id.split('_')[0], axis=1)
meta_df.drop('folder_id',inplace=True,axis=1)
meta_df.drop('study_id',inplace=True,axis=1)
meta_df['dim0'] = meta_df['height']
meta_df['dim1'] = meta_df['width']
meta_df.drop('width',inplace=True,axis=1)
meta_df.drop('height',inplace=True,axis=1)
df = df.merge(meta_df, on='id',how="left")

## Preprocesing the images off-line
In this case we apply the preprocesing (histogram equalization and CLAHE in an off-line fashio due to its easier to implement for YOLO model)

In [17]:
BASE_PATH_TRAIN = '/kaggle/working/dataset/images/train'
BASE_PATH_VAL = '/kaggle/working/dataset/images/val'

clahe = cv2.createCLAHE(clipLimit=2.0)
for image in tqdm(os.listdir(BASE_PATH_TRAIN)):
    full_image_path = os.path.join(BASE_PATH_TRAIN,image)
    img = cv2.imread(full_image_path,cv2.IMREAD_UNCHANGED)
    img_eq = cv2.equalizeHist(img)
    img_clahe = clahe.apply(img_eq)
    cv2.imwrite(full_image_path,img_clahe)

100%|██████████| 3435/3435 [00:31<00:00, 107.76it/s]


In [18]:
for image in tqdm(os.listdir(BASE_PATH_VAL)):
    full_image_path = os.path.join(BASE_PATH_VAL,image)
    img = cv2.imread(full_image_path,cv2.IMREAD_UNCHANGED)
    img_eq = cv2.equalizeHist(img)
    img_clahe = clahe.apply(img_eq)
    cv2.imwrite(full_image_path,img_clahe)

100%|██████████| 859/859 [00:07<00:00, 107.96it/s]


## Labels Generation

### Training Labels

In [20]:
for i,image in tqdm(enumerate(train_image_paths)):
    image_id = image.split('_')[0]
    row = df[df['id']==image_id].iloc[0]
    label = row.image_level
    if label=='opacity':
        bboxes = get_bbox(row)
        scale_bboxes = scale_bbox(row,bboxes)
        yolo_bboxes = get_yolo_format_bbox(IMG_SIZE,IMG_SIZE,scale_bboxes)
        with (open(f'/kaggle/working/dataset/labels/train/{image_id}.txt','w')) as file:
            for bbox in yolo_bboxes:
                bbox = [0]+bbox
                bbox = [str(i) for i in bbox]
                bbox = ' '.join(bbox)
                file.write(bbox)
                file.write('\n')
                

3435it [00:03, 940.08it/s]


### Validation Labels

In [21]:
for i,image in tqdm(enumerate(val_image_paths)):
    image_id = image.split('_')[0]
    row = df[df['id']==image_id].iloc[0]
    label = row.image_level
    if label=='opacity':
        bboxes = get_bbox(row)
        scale_bboxes = scale_bbox(row,bboxes)
        yolo_bboxes = get_yolo_format_bbox(IMG_SIZE,IMG_SIZE,scale_bboxes)
        with (open(f'/kaggle/working/dataset/labels/val/{image_id}.txt','w')) as file:
            for bbox in yolo_bboxes:
                bbox = [0]+bbox
                bbox = [str(i) for i in bbox]
                bbox = ' '.join(bbox)
                file.write(bbox)
                file.write('\n')

859it [00:00, 921.04it/s]


## Creating the hyperparameters YAML file

In [22]:
hyp_yaml = dict(
    lr0=0.002,
    lrf=0.02,
    momentum=0.9,
    weight_decay=0.0005,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=0.05,
    cls=0.5,
    cls_pw=1.0,
    obj=1.0,
    obj_pw=1.0,
    iou_t=0.2,
    anchor_t=4.0,
    fl_gamma=0.0,
    hsv_h=0,
    hsv_s=0,
    hsv_v=0,
    degrees=0,
    translate=0,
    scale=0,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0,
    mosaic=0,
    mixup=0.0,
    copy_paste=0.0
)

with open('/kaggle/working/hyp.yaml','w') as file:
    yaml.dump(hyp_yaml,file,default_flow_style=True)

In [23]:
%cd yolov5

/kaggle/working/yolov5


In [24]:
name = f'{EXP_NAME}_{dt_string}'

In [ ]:
!python train.py --img 512 \
                 --batch 16 \
                 --epochs {EPOCHS} \
                 --data data.yaml \
                 --weights yolov5x.pt \
                 --save-period 50\
                 --project Final-Covid19-Detection --hyp /kaggle/working/hyp.yaml --name {name}

wandb: Currently logged in as: alvaromoureupm. Use `wandb login --relogin` to force relogin
train: weights=yolov5x.pt, cfg=, data=data.yaml, hyp=/kaggle/working/hyp.yaml, epochs=300, batch_size=16, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=Final-Covid19-Detection, name=yoloV5x_300e_fold0_04/11_20:52, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=50, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 3987, done.
remote: Counting objects: 100% (979/979), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 3987 (delta 965), reused 968 (delta 962), pack-reused 3008
Receiving objects: 100% (3987/3987), 1.19 MiB | 17.92 MiB/s, done.
Resolving 